In [209]:
# Code to import the .xlsx file.

import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf8')

# Isabelle: change the name below to the name of the excel file you're importing. Make sure it's in the same directory
df = pd.read_excel('FactBase_0421.xlsx', index_col=None)
df.fillna(-1, inplace=True)
df = df.apply(lambda x: x.astype(str).str.lower())
df.replace("additional accounting", "additional attribute", inplace=True)
df.replace("additional reporting", "additional attribute", inplace=True)

df

In [210]:
df

,BG,BU,SourceSystem,GL Major Product,PRKey,FeedType,Typename,FinancialReportingRequirements,SumofAttributesUsed,Reqcategory,...,Attribute12_AttributeName,Attribute12_AttributeDefinition_ifnetnewdataattribute,Attribute12_StrategytoObtainAttribute,Attribute12_Ifattributewillbederivedhasbusinesslogicbeendefined,Attribute12_Ifattributewillbederivedwhatreferencedatamappingtablesarerequired,Attribute12_DataType,Attribute12_IfDatewhatisLayout,Attribute12_DataLength,Attribute12_Nullable,Attribute12_Concatenated
0,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,portfolio,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
1,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,pas system identifier,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
2,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,how to uniquely identify contract under ifrs 1...,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
3,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,insured vs non-insured,-1.0,additional attribute,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
4,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,date contract was issued (i.e. policy / benefi...,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
5,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,cohort / group,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
6,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,cash flow indicator (cash / accrual / adjustme...,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
7,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,"account: premiums (first year, renewal, single)",-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
8,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,"account: claims (e.g. death, annuity, etc.)",-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1
9,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new file layout,new gb ebf,account: investment component (how you identif...,-1.0,ifrs 17,...,-1,-1,-1,-1,-1.0,-1,-1.0,-1.0,-1,-1


In [211]:
# This is a check to see the number of unique PR keys in the file. Should be 171, unless we add / remove from iteration 3
df1 = df
len(df1.loc[ : , 'PRKey' ].unique())

171

In [248]:
# Isbelle: This is the name of the output file. Change below to what you want the output file to be, you run this block TWICE.
# Once at the beginning, and then again at the end.
writer = pd.ExcelWriter('Output0421.xlsx', engine='xlsxwriter')

In [213]:
# This block removes all of the reference data feeds, since iteration 3 was only analysis on transaction / event feeds

df = df[~df.loc[ : , 'PRKey' ].str.contains("ref")]
df = df[~df.loc[ : , 'PRKey' ].str.contains(".prn")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - chess service billing")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("data mart")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - pheonix")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - residence distribution")]
df = df[~df.loc[ : , 'PRKey' ].str.contains("gf.txt - bif, s&c")]

# This is a check to see the count of unique PR Keys after we remove all the reference data. It should be 133, unless we have
# added or removed feeds

len(df.loc[ : , 'PRKey' ].unique())

133

In [214]:
# This sums the count of feeds using a particular strategy

summary = pd.DataFrame({'Feed Count': df.groupby(['Reqcategory','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements'])['BUProposedStrategytoMeetRequirements'].count()}).reset_index()
summary_file = pd.DataFrame({'Feed Count': df.groupby(['Reqcategory','Typename','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements'])['BUProposedStrategytoMeetRequirements'].count()}).reset_index()

In [215]:
# This creates the table with all the files using a particular file type, by BG and BU

file_types_all = pd.DataFrame(df.groupby(['Typename', 'BG', 'BU','PRKey'])['Typename'].count())


In [216]:
# This creates the table with the count and percentage of all IFRS 17 attributes that are not "undecided"

summary_derived = df[(df['Reqcategory']=='ifrs 17') & (df['BUProposedStrategytoMeetRequirements']!='undecided')]
bu_strategy = pd.DataFrame({'# Attrb Decided': summary_derived.groupby(['BG', 'BU','PRKey'])['PRKey'].count()}).reset_index()
bu_strategy['Percentage Decided'] = bu_strategy['# Attrb Decided'] / 17 * 100
bu_strategy


,BG,BU,PRKey,# Attrb Decided,Percentage Decided
0,asia,all bu,tbd (sun bu (mje) to ah),16,94.117647
1,asia,csl,pamcs002.dat,16,94.117647
2,asia,csl,pfmcs002.dat,16,94.117647
3,asia,csl,tbd (snaps to aah),16,94.117647
4,asia,slamci,mrsmf001.dat,16,94.117647
5,asia,slamci,tpsmf001.dat,16,94.117647
6,asia,slfi,acpgx003.dat,16,94.117647
7,asia,slfi,ingio001.dat,16,94.117647
8,asia,slfi,pamio002.dat,16,94.117647
9,asia,slfi,pfmio002.dat,16,94.117647


In [217]:
# This lists out all the unique file type names in the document

file_types = df.Typename.unique()

file_types

array(['new gb ebf', 'new gb aso tbf', 'new us tbf', 'frvlj', 'direct',
       'modified frvlj', 'poorman', 'modified direct', 'aeb',
       'new grs ebf', 'pseudo-event', 'ais', 'sae', 'lawson', 'pre-direct',
       'mfs', 'modified sl', 'hrgx0001', 'sl format'], dtype=object)

In [218]:
# This lists and counts the total number of attributes per file type 
attributes_file = df.FinancialReportingRequirements.unique()
attributes_file_count = {}
for j in file_types:
    nested = {}
    for i in attributes_file:
        name = df.loc[(df['Typename'].isin([j])) & (df['FinancialReportingRequirements'].isin([i]))]
        nested[i] = name.FinancialReportingRequirements.count()
    attributes_file_count[j] = nested

attributes_file_count

{'aeb': {'general ledger base amount': 0,
  'code': 0,
  'bank disbursement number': 1,
  'record type': 0,
  'control total recs': 1,
  'cusip_value': 0,
  'investment component': 0,
  'mail cheque to': 0,
  'control id': 1,
  'client number': 0,
  'federal tax': 0,
  'reinsurance: date of initial recognition of cession': 2,
  'program run time': 0,
  'description text': 0,
  'fund manager': 0,
  'frvlju01-tpa-name': 0,
  'in-source-doc-ref': 1,
  'billing frequency': 0,
  'on-sales-tax': 0,
  'in-jrnl-id-location': 1,
  'creation date of the payment': 0,
  'tpa state': 0,
  'mdb indicator': 0,
  'prevsegment': 0,
  'accounting unit': 0,
  'transmission hour': 0,
  'event cd': 0,
  'trans-amt': 0,
  'trans trans_subtype': 0,
  'gross incurral': 0,
  'telephone num': 0,
  'segval10': 0,
  'bank number': 0,
  'cheque request - payee address 3': 0,
  'cheque request - payee address 2': 0,
  'cheque request - payee address 1': 0,
  'bene-sub-type': 0,
  'mess id': 0,
  'claim type': 0,
  

In [219]:
# This creates the "percentage" column that shows you total percentage of each attribute using one strategy to send the attribute
# by file type
summary_file['Percentage'] = ""
for file_type in attributes_file_count:
    for key in attributes_file_count[file_type]:
        new = summary_file.loc[(summary_file['Typename'].isin([file_type])) & (summary_file['FinancialReportingRequirements'].isin([key]))]
        new['Percentage'] = new['Feed Count'] / attributes_file_count[file_type][key] * 100
        summary_file.update(new)

summary_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage
0,additional attribute,aeb,accounting key,attribute in feed,1.0,100
1,additional attribute,aeb,agent number,attribute in feed,1.0,100
2,additional attribute,aeb,amount class 1,attribute in feed,1.0,100
3,additional attribute,aeb,amount class 2,attribute in feed,1.0,100
4,additional attribute,aeb,block of business,attribute in feed,1.0,100
5,additional attribute,aeb,change stamp,attribute in feed,1.0,100
6,additional attribute,aeb,city name,attribute in feed,1.0,100
7,additional attribute,aeb,control credit amount,attribute in feed,1.0,100
8,additional attribute,aeb,control credit recs,attribute in feed,1.0,100
9,additional attribute,aeb,control date,attribute in feed,1.0,100


In [220]:
# This shows the maximum percentage of 1 attribute leveraging 1 strategy to send an attribute, by file type

summary_file['Strat Percent Max'] = summary_file.groupby(['Typename','FinancialReportingRequirements'])['Percentage'].transform(max)
summary_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max
0,additional attribute,aeb,accounting key,attribute in feed,1.0,100,100.000000
1,additional attribute,aeb,agent number,attribute in feed,1.0,100,100.000000
2,additional attribute,aeb,amount class 1,attribute in feed,1.0,100,100.000000
3,additional attribute,aeb,amount class 2,attribute in feed,1.0,100,100.000000
4,additional attribute,aeb,block of business,attribute in feed,1.0,100,100.000000
5,additional attribute,aeb,change stamp,attribute in feed,1.0,100,100.000000
6,additional attribute,aeb,city name,attribute in feed,1.0,100,100.000000
7,additional attribute,aeb,control credit amount,attribute in feed,1.0,100,100.000000
8,additional attribute,aeb,control credit recs,attribute in feed,1.0,100,100.000000
9,additional attribute,aeb,control date,attribute in feed,1.0,100,100.000000


In [221]:
# This lists and counts the number of unique attributes being sent / derived by a feed

attributes = df.FinancialReportingRequirements.unique()
attributes
attributes_count = {}
for i in attributes:
    name = df.loc[df['FinancialReportingRequirements'].isin([i])]
    attributes_count[i] = name.FinancialReportingRequirements.count()

attributes_count

{'general ledger base amount': 1,
 'code': 1,
 'bank disbursement number': 116,
 'record type': 4,
 'control total recs': 1,
 'cusip_value': 1,
 'investment component': 1,
 'mail cheque to': 2,
 'control id': 1,
 'client number': 1,
 'federal tax': 3,
 'reinsurance: date of initial recognition of cession': 133,
 'program run time': 1,
 'description text': 1,
 'fund manager': 2,
 'frvlju01-tpa-name': 1,
 'in-source-doc-ref': 1,
 'billing frequency': 26,
 'on-sales-tax': 1,
 'in-jrnl-id-location': 1,
 'creation date of the payment': 2,
 'tpa state': 2,
 'mdb indicator': 1,
 'prevsegment': 1,
 'accounting unit': 1,
 'transmission hour': 3,
 'event cd': 1,
 'trans-amt': 1,
 'trans trans_subtype': 1,
 'gross incurral': 1,
 'telephone num': 3,
 'segval10': 1,
 'bank number': 4,
 'cheque request - payee address 3': 3,
 'cheque request - payee address 2': 3,
 'cheque request - payee address 1': 3,
 'bene-sub-type': 1,
 'mess id': 1,
 'claim type': 4,
 'cheque request - approved by id': 3,
 'cl

In [222]:
# This shows the number of attributes leveraging the same strategy to send data point to AAH, as well as the total percentage
# out of all possible strategies

summary['Percentage'] = ""
for key in attributes_count:
    new = summary.loc[(summary['FinancialReportingRequirements'].isin([key]))]
    new['Percentage'] = new['Feed Count'] / attributes_count[key] * 100
    summary.update(new)

summary

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage
0,additional attribute,1st billing pause,attribute in feed,2.0,100
1,additional attribute,2nd billing pause,attribute in feed,2.0,100
2,additional attribute,abc account number,attribute in feed,8.0,100
3,additional attribute,account code - master,attribute in feed,1.0,100
4,additional attribute,account code - sub,attribute in feed,1.0,100
5,additional attribute,account currency code,attribute in feed,1.0,100
6,additional attribute,account name,attribute in feed,1.0,100
7,additional attribute,account num,attribute in feed,1.0,100
8,additional attribute,account reference,attribute in feed,3.0,100
9,additional attribute,account-crncy,attribute in feed,1.0,100


In [223]:
# This shows the maximum percentage for 1 attribute leveraging 1 strategy to send to AAH

summary['Strat Percent Max'] = summary.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)
summary

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max
0,additional attribute,1st billing pause,attribute in feed,2.0,100,100.000000
1,additional attribute,2nd billing pause,attribute in feed,2.0,100,100.000000
2,additional attribute,abc account number,attribute in feed,8.0,100,100.000000
3,additional attribute,account code - master,attribute in feed,1.0,100,100.000000
4,additional attribute,account code - sub,attribute in feed,1.0,100,100.000000
5,additional attribute,account currency code,attribute in feed,1.0,100,100.000000
6,additional attribute,account name,attribute in feed,1.0,100,100.000000
7,additional attribute,account num,attribute in feed,1.0,100,100.000000
8,additional attribute,account reference,attribute in feed,3.0,100,100.000000
9,additional attribute,account-crncy,attribute in feed,1.0,100,100.000000


In [224]:
# This shows the unique strategies to derive an IFRS 17 data point, by file type

is_derived_file =  df['BUProposedStrategytoMeetRequirements']=='derived from feed attributes'
summary_derived_file = df[is_derived_file]

total_derived_file = summary_derived_file.groupby(['Typename','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived_file = total_derived_file[['Typename','FinancialReportingRequirements', 'Total_Derived']]
total_derived_file
duplicates_file = summary_derived_file.groupby(['Typename','Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col_file = pd.merge(duplicates_file, total_derived_file, on=['Typename','FinancialReportingRequirements'])
df_merge_col_file['Percentage'] = df_merge_col_file['Count'] / df_merge_col_file['Total_Derived'] * 100
df_merge_col_file


,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,aeb,gl accounting requirements,"accounting bases of feeds (i.e. ifrs 4, ifrs 1...",derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
1,aeb,gl accounting requirements,inter-company indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
2,aeb,gl accounting requirements,reinsurance type indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
3,aeb,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
4,aeb,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
5,aeb,ifrs 17,cohort / group,derived from feed attributes,system id,source id,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000
6,aeb,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
7,aeb,ifrs 17,portfolio,derived from feed attributes,in-jrnl-id-sys-id,in-jrnl-id-source,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000
8,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
9,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000


In [225]:
# This produces the maximum percentage of all different strategies to derive an IFRS 17 reporting requirement, in 1 file type

df_merge_col_file['Derived Percent Max'] = df_merge_col_file.groupby(['Typename','FinancialReportingRequirements'])['Percentage'].transform(max)
df_merge_col_file


,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,aeb,gl accounting requirements,"accounting bases of feeds (i.e. ifrs 4, ifrs 1...",derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
1,aeb,gl accounting requirements,inter-company indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
2,aeb,gl accounting requirements,reinsurance type indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
3,aeb,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
4,aeb,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
5,aeb,ifrs 17,cohort / group,derived from feed attributes,system id,source id,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000,100.000000
6,aeb,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
7,aeb,ifrs 17,portfolio,derived from feed attributes,in-jrnl-id-sys-id,in-jrnl-id-source,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000,100.000000
8,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
9,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000


In [226]:
# Lists out all the attributes being used to derive an IFRS 17 attribute across all file types, as well as total count / percentage 
# leveraging the same strategy

is_derived =  df['BUProposedStrategytoMeetRequirements']=='derived from feed attributes'
summary_derived = df[is_derived]
summary_derived
total_derived = summary_derived.groupby(['FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived = total_derived[['FinancialReportingRequirements', 'Total_Derived']]
total_derived
duplicates = summary_derived.groupby(['Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col = pd.merge(duplicates, total_derived, on=['FinancialReportingRequirements'])
df_merge_col
df_merge_col['Percentage'] = df_merge_col['Count'] / df_merge_col['Total_Derived'] * 100
df_merge_col

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,additional attribute,accouting period,derived from feed attributes,app date,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
1,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000
2,additional attribute,channel,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
3,additional attribute,corporation,derived from feed attributes,issue,line of business,transaction type,account number,-1,-1,-1,-1,-1,-1,1,1,100.000000
4,additional attribute,deduction amount,derived from feed attributes,deduction amount,deduction code,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
5,additional attribute,desc2_part1,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
6,additional attribute,desc2_part2,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
7,additional attribute,future,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
8,additional attribute,legacy system key,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
9,additional attribute,legacy system reference key,derived from feed attributes,corp,segment,account,cost center,tax,issue,residence,source row number,-1,-1,2,2,100.000000


In [227]:
# Part of sense check to ensure there are no duplicate strategies to derive IFRS 17 attributes
attribute_list_ifrs17= df_merge_col[ df_merge_col['Reqcategory']=="ifrs 17"]
attribute_list_ifrs17

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
80,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,34,42,80.952381
81,ifrs 17,"account: claims (e.g. death, annuity, etc.)",derived from feed attributes,claims,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,42,19.047619
82,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,34,5.882353
83,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,32,34,94.117647
84,ifrs 17,account: investment component (how you identif...,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,28,29,96.551724
85,ifrs 17,account: investment component (how you identif...,derived from feed attributes,guaranteed indicator,guaranteed start date,guaranteed end date,death benefit,-1,-1,-1,-1,-1,-1,1,29,3.448276
86,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,41,47,87.234043
87,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,participation number,prior renewal effective date,rate guarantee date,cash vs accrued indicator,source system id,benefit available date,policy source 1,policy source 2,policy source 3,policy source 4,4,47,8.510638
88,ifrs 17,"account: premiums (first year, renewal, single)",derived from feed attributes,sub category code,benefit name,bill effective date,benefit effective date,-1,-1,-1,-1,-1,-1,2,47,4.255319
89,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,48,6.250000


In [228]:
# Part of sense check to ensure there are no duplicate strategies to derive IFRS 17 attributes. If there are no duplicates,
# this will produce an empty table. If there are duplicates, they will be listed out on the table below

attribute_list = df_merge_col.iloc[:,0:13]
attribute_list = attribute_list[attribute_list['Reqcategory']=="ifrs 17"]

all_ifrs17 = attribute_list.FinancialReportingRequirements.unique()
attribute_list.drop("BUProposedStrategytoMeetRequirements", axis=1, inplace=True)
attribute_list.drop("Reqcategory", axis=1, inplace=True)

list_of_variation = attribute_list.values.tolist()
list_of_variation

duplicates = {}
counter = 0
for attrb in all_ifrs17:
    attribute_list_1 = attribute_list[attribute_list['FinancialReportingRequirements']==attrb]
    list_of_variation = attribute_list_1.values.tolist()
    for i in range(len(list_of_variation) - 1):
        set_of_dups = []
        if list_of_variation[i][0] not in duplicates:
            duplicates[list_of_variation[i][0]] = set_of_dups
        for j in range(i+1, len(list_of_variation)):
            if(set(list_of_variation[i]) == (set(list_of_variation[j]))):
                set_of_dups.append(str(list_of_variation[i][1:]))
                set_of_dups.append(str(list_of_variation[j][1:]))
                counter +=1
        if len(set_of_dups) > 0:
            duplicates[list_of_variation[i][0]].append(set_of_dups)
            
duplicates_1 = pd.DataFrame.from_dict(duplicates,orient='index').transpose()
duplicates_1

,expense indicator to distinguish between maintenance/acquisition and ifrs 17 attributable/non-attributable expenses,"account: claims (e.g. death, annuity, etc.)","cash flow indicator (cash / accrual / adjustment, etc.)",account: directly attributable acquisition costs,account: investment component (how you identify the investment component of a claim/benefit transaction),how to uniquely identify contract under ifrs 17 ? (i.e. policy / benefit number),portfolio,"account: premiums (first year, renewal, single)",pas system identifier,cohort / group
0,None,None,"[['pr key', 'line of business', '-1', '-1', '-...",None,None,None,None,None,None,None


In [229]:
# Isabelle: This creates a csv of the table containing duplicate strategies to derive IFRS 17 reporting requirements. 
# Only run this block if the table above is not empty. Then go into the .csv created, and you will need to manually 
# investigate the duplicates in the master fact base file

duplicates_1.to_csv("duplicates_derived.csv")

In [230]:
#  Creates max percentage of attributes using one strategy to derive
df_merge_col['Derived Percent Max'] = df_merge_col.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)

df_merge_col

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,additional attribute,accouting period,derived from feed attributes,app date,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
1,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,6,100.000000,100.000000
2,additional attribute,channel,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
3,additional attribute,corporation,derived from feed attributes,issue,line of business,transaction type,account number,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
4,additional attribute,deduction amount,derived from feed attributes,deduction amount,deduction code,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
5,additional attribute,desc2_part1,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
6,additional attribute,desc2_part2,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
7,additional attribute,future,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
8,additional attribute,legacy system key,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
9,additional attribute,legacy system reference key,derived from feed attributes,corp,segment,account,cost center,tax,issue,residence,source row number,-1,-1,2,2,100.000000,100.000000


In [231]:
# Part of code to produce max derived by percentage by file type 
sliced_derived = df_merge_col[["FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements", "Derived Percent Max"]]
sliced_derived = sliced_derived.drop_duplicates()

In [232]:
# Part of code to produce max derived by percentage by file type 

total_derived_file = summary_derived.groupby(['Typename', 'FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements']).size().reset_index(name='Total_Derived')
total_derived_f = total_derived_file[['Typename','FinancialReportingRequirements', 'Total_Derived']]
total_derived_f
duplicates_file = summary_derived.groupby(['Typename','Reqcategory', 'FinancialReportingRequirements', "BUProposedStrategytoMeetRequirements",'Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']).size().reset_index(name='Count')
df_merge_col_file = pd.merge(duplicates_file, total_derived_f, on=['Typename','FinancialReportingRequirements'])
df_merge_col_file['Percentage'] = df_merge_col_file['Count'] / df_merge_col_file['Total_Derived'] * 100
df_merge_col_file

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage
0,aeb,gl accounting requirements,"accounting bases of feeds (i.e. ifrs 4, ifrs 1...",derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
1,aeb,gl accounting requirements,inter-company indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
2,aeb,gl accounting requirements,reinsurance type indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
3,aeb,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
4,aeb,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
5,aeb,ifrs 17,cohort / group,derived from feed attributes,system id,source id,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000
6,aeb,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000
7,aeb,ifrs 17,portfolio,derived from feed attributes,in-jrnl-id-sys-id,in-jrnl-id-source,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000
8,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000
9,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000


In [233]:
# Part of code to produce max derived by percentage by file type 

df_merge_col_file['Derived Percent Max'] = df_merge_col_file.groupby(['FinancialReportingRequirements', "Typename"])['Percentage'].transform(max)
df_merge_col_file

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Attribute1_AttributeDefinition_ifnetnewdataattribute,Attribute2_AttributeDefinition_ifnetnewdataattribute,Attribute3_AttributeDefinition_ifnetnewdataattribute,Attribute4_AttributeDefinition_ifnetnewdataattribute,Attribute5_AttributeDefinition_ifnetnewdataattribute,Attribute6_AttributeDefinition_ifnetnewdataattribute,Attribute7_AttributeDefinition_ifnetnewdataattribute,Attribute8_AttributeDefinition_ifnetnewdataattribute,Attribute9_AttributeDefinition_ifnetnewdataattribute,Attribute10_AttributeDefinition_ifnetnewdataattribute,Count,Total_Derived,Percentage,Derived Percent Max
0,aeb,gl accounting requirements,"accounting bases of feeds (i.e. ifrs 4, ifrs 1...",derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
1,aeb,gl accounting requirements,inter-company indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
2,aeb,gl accounting requirements,reinsurance type indicator,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
3,aeb,ifrs 17,account: directly attributable acquisition costs,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
4,aeb,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,account,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
5,aeb,ifrs 17,cohort / group,derived from feed attributes,system id,source id,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000,100.000000
6,aeb,ifrs 17,expense indicator to distinguish between maint...,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,100.000000,100.000000
7,aeb,ifrs 17,portfolio,derived from feed attributes,in-jrnl-id-sys-id,in-jrnl-id-source,policy number,in-primary,in-secondary,in-type,-1,-1,-1,-1,1,1,100.000000,100.000000
8,ais,additional attribute,association of nonpar riders with the base plan,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,2,100.000000,100.000000
9,ais,additional attribute,minor product,derived from feed attributes,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,100.000000,100.000000


In [234]:
# Shows percentage of feeds deriving IFRS 17 reporting requirements, by file type

sliced_derived_file = df_merge_col_file[["Typename", "FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements", "Derived Percent Max"]]
sliced_derived_file = sliced_derived_file.drop_duplicates()
sliced_derived_file

,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Derived Percent Max
0,aeb,"accounting bases of feeds (i.e. ifrs 4, ifrs 1...",derived from feed attributes,100.000000
1,aeb,inter-company indicator,derived from feed attributes,100.000000
2,aeb,reinsurance type indicator,derived from feed attributes,100.000000
3,aeb,account: directly attributable acquisition costs,derived from feed attributes,100.000000
4,aeb,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,100.000000
5,aeb,cohort / group,derived from feed attributes,100.000000
6,aeb,expense indicator to distinguish between maint...,derived from feed attributes,100.000000
7,aeb,portfolio,derived from feed attributes,100.000000
8,ais,association of nonpar riders with the base plan,derived from feed attributes,100.000000
9,ais,minor product,derived from feed attributes,100.000000


In [235]:
# Strategy to produce attribute by file type
strat_and_derived_file = pd.merge(summary_file, sliced_derived_file, on=["Typename","BUProposedStrategytoMeetRequirements","FinancialReportingRequirements"],how="outer")
strat_and_derived_file

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max,Derived Percent Max
0,additional attribute,aeb,accounting key,attribute in feed,1.0,100,100.000000,NaN
1,additional attribute,aeb,agent number,attribute in feed,1.0,100,100.000000,NaN
2,additional attribute,aeb,amount class 1,attribute in feed,1.0,100,100.000000,NaN
3,additional attribute,aeb,amount class 2,attribute in feed,1.0,100,100.000000,NaN
4,additional attribute,aeb,block of business,attribute in feed,1.0,100,100.000000,NaN
5,additional attribute,aeb,change stamp,attribute in feed,1.0,100,100.000000,NaN
6,additional attribute,aeb,city name,attribute in feed,1.0,100,100.000000,NaN
7,additional attribute,aeb,control credit amount,attribute in feed,1.0,100,100.000000,NaN
8,additional attribute,aeb,control credit recs,attribute in feed,1.0,100,100.000000,NaN
9,additional attribute,aeb,control date,attribute in feed,1.0,100,100.000000,NaN


In [236]:
strat_and_derived = pd.merge(summary, sliced_derived, on=["BUProposedStrategytoMeetRequirements","FinancialReportingRequirements"],how="outer")
strat_and_derived

,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat Percent Max,Derived Percent Max
0,additional attribute,1st billing pause,attribute in feed,2.0,100,100.000000,NaN
1,additional attribute,2nd billing pause,attribute in feed,2.0,100,100.000000,NaN
2,additional attribute,abc account number,attribute in feed,8.0,100,100.000000,NaN
3,additional attribute,account code - master,attribute in feed,1.0,100,100.000000,NaN
4,additional attribute,account code - sub,attribute in feed,1.0,100,100.000000,NaN
5,additional attribute,account currency code,attribute in feed,1.0,100,100.000000,NaN
6,additional attribute,account name,attribute in feed,1.0,100,100.000000,NaN
7,additional attribute,account num,attribute in feed,1.0,100,100.000000,NaN
8,additional attribute,account reference,attribute in feed,3.0,100,100.000000,NaN
9,additional attribute,account-crncy,attribute in feed,1.0,100,100.000000,NaN


In [237]:
# Analysis for data type

all_definitions = df[df['BUProposedStrategytoMeetRequirements']=='attribute in feed']
count_all_definitions = pd.DataFrame({'Total Feeds': all_definitions.groupby(['Reqcategory','FinancialReportingRequirements'])['FinancialReportingRequirements'].count()}).reset_index()

count_definitions = pd.DataFrame({'Feeds Using Datatype': all_definitions.groupby(['Reqcategory','FinancialReportingRequirements', 'BUProposedStrategytoMeetRequirements', 'DataType'])['DataType'].count()}).reset_index()
        
data_definitions = pd.merge(count_all_definitions, count_definitions, on=['Reqcategory',"FinancialReportingRequirements"],how="outer")

data_definitions['Percentage'] = data_definitions['Feeds Using Datatype'] / data_definitions['Total Feeds'] * 100
data_definitions

,Reqcategory,FinancialReportingRequirements,Total Feeds,BUProposedStrategytoMeetRequirements,DataType,Feeds Using Datatype,Percentage
0,additional attribute,1st billing pause,2,attribute in feed,-1,1,50.000000
1,additional attribute,1st billing pause,2,attribute in feed,varchar,1,50.000000
2,additional attribute,2nd billing pause,2,attribute in feed,-1,1,50.000000
3,additional attribute,2nd billing pause,2,attribute in feed,varchar,1,50.000000
4,additional attribute,abc account number,8,attribute in feed,alphanumeric,8,100.000000
5,additional attribute,account code - master,1,attribute in feed,numeric,1,100.000000
6,additional attribute,account code - sub,1,attribute in feed,numeric,1,100.000000
7,additional attribute,account currency code,1,attribute in feed,int,1,100.000000
8,additional attribute,account name,1,attribute in feed,character,1,100.000000
9,additional attribute,account num,1,attribute in feed,-1,1,100.000000


In [238]:
# Percentage of attributes using same data type

data_definitions['Data Type Percent Max'] = data_definitions.groupby(['FinancialReportingRequirements'])['Percentage'].transform(max)

data_definitions_slice= data_definitions[["FinancialReportingRequirements", 'BUProposedStrategytoMeetRequirements', "Data Type Percent Max"]].drop_duplicates()
strat_derived_dtype = pd.merge(strat_and_derived, data_definitions_slice, on=["FinancialReportingRequirements", "BUProposedStrategytoMeetRequirements"],how="outer")
strat_derived_dtype.drop("Percentage", axis=1, inplace=True)

In [185]:
# Recommendations for derived

df_merge_col_file_rec = df_merge_col_file[(df_merge_col_file['Reqcategory']=='ifrs 17')]
df_merge_col_file_rec['Proposed_Attributes_To_Derive'] = df_merge_col_file_rec[['Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute']].apply(lambda x: ', '.join(x), axis=1)
df_merge_col_file_rec.drop(['Attribute1_AttributeDefinition_ifnetnewdataattribute','Attribute2_AttributeDefinition_ifnetnewdataattribute','Attribute3_AttributeDefinition_ifnetnewdataattribute','Attribute4_AttributeDefinition_ifnetnewdataattribute','Attribute5_AttributeDefinition_ifnetnewdataattribute','Attribute6_AttributeDefinition_ifnetnewdataattribute','Attribute7_AttributeDefinition_ifnetnewdataattribute','Attribute8_AttributeDefinition_ifnetnewdataattribute','Attribute9_AttributeDefinition_ifnetnewdataattribute','Attribute10_AttributeDefinition_ifnetnewdataattribute'], axis=1, inplace=True)
df_merge_col_file_rec = df_merge_col_file_rec[['Typename', 'Reqcategory','FinancialReportingRequirements','BUProposedStrategytoMeetRequirements',                      'Proposed_Attributes_To_Derive','Count',
 'Total_Derived','Percentage','Derived Percent Max']]
df_merge_col_file_rec['Proposed_Attributes_To_Derive'] = df_merge_col_file_rec.Proposed_Attributes_To_Derive.str.replace('-1' ,'')
df_merge_col_file_rec.rename(columns={'Derived Percent Max':'Derived_Percent_Max'}, inplace=True)
df_merge_col_file_rec['Majority_Attributes_To_Derive'] = df_merge_col_file_rec[df_merge_col_file_rec.Percentage.values == df_merge_col_file_rec.Derived_Percent_Max.values]["Proposed_Attributes_To_Derive"]
    
df_merge_col_file_rec

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Proposed_Attributes_To_Derive,Count,Total_Derived,Percentage,Derived_Percent_Max,Majority_Attributes_To_Derive
0,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , .0, .0, .0, .0",3,6,50.000000,50.000000,"pr key, line of business, , , , , .0, .0, .0, .0"
1,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, line of business, li...",1,6,16.666667,50.000000,NaN
2,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, test2, line of business, test3, , , .0...",1,6,16.666667,50.000000,NaN
3,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"test1, line of business, line of business, lin...",1,6,16.666667,50.000000,NaN
4,direct,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, te...",1,1,100.000000,100.000000,"gross benefit, federal tax, provincial tax, te..."
5,mfs,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, te...",1,1,100.000000,100.000000,"gross benefit, federal tax, provincial tax, te..."
6,new gb ebf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
7,new gb ebf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"pas system identifier, , , , , , .0, .0, .0, .0"
8,new us tbf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , .0, .0, .0, .0",1,3,33.333333,66.666667,NaN
9,new us tbf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"tbd, , , , , , .0, .0, .0, .0",2,3,66.666667,66.666667,"tbd, , , , , , .0, .0, .0, .0"


In [239]:
df_merge_col_file_rec['Majority_Attributes_To_Derive'] = df_merge_col_file_rec.groupby(['Typename', 'FinancialReportingRequirements'])['Majority_Attributes_To_Derive'].transform('first')
df_merge_col_file_rec

,Typename,Reqcategory,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Proposed_Attributes_To_Derive,Count,Total_Derived,Percentage,Derived_Percent_Max,Majority_Attributes_To_Derive
0,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , .0, .0, .0, .0",3,6,50.000000,50.000000,"pr key, line of business, , , , , .0, .0, .0, .0"
1,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, line of business, li...",1,6,16.666667,50.000000,"pr key, line of business, , , , , .0, .0, .0, .0"
2,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, test2, line of business, test3, , , .0...",1,6,16.666667,50.000000,"pr key, line of business, , , , , .0, .0, .0, .0"
3,ais,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"test1, line of business, line of business, lin...",1,6,16.666667,50.000000,"pr key, line of business, , , , , .0, .0, .0, .0"
4,direct,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, te...",1,1,100.000000,100.000000,"gross benefit, federal tax, provincial tax, te..."
5,mfs,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, te...",1,1,100.000000,100.000000,"gross benefit, federal tax, provincial tax, te..."
6,new gb ebf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
7,new gb ebf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
8,new us tbf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"pr key, line of business, , , , , .0, .0, .0, .0",1,3,33.333333,66.666667,"tbd, , , , , , .0, .0, .0, .0"
9,new us tbf,ifrs 17,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,"tbd, , , , , , .0, .0, .0, .0",2,3,66.666667,66.666667,"tbd, , , , , , .0, .0, .0, .0"


In [240]:
summary_file_rec = summary_file[(summary_file['Reqcategory']=='ifrs 17')]
summary_file_rec.rename(columns={'Strat Percent Max':'Strat_Percent_Max'}, inplace=True)
# summary_file_rec['Strat_Reccomendation'] = summary_file_rec.groupby('FinancialReportingRequirements')['Strat_Reccomendation'].transform('first')
summary_file_rec
# Percentage for strategy that is not "not required" or "undecided"

summary_file_rec['Real_Strategy'] = summary_file_rec[(summary_file_rec["BUProposedStrategytoMeetRequirements"] != "undecided") & (summary_file_rec["BUProposedStrategytoMeetRequirements"] != "attribute not required")].groupby(['Typename', 'FinancialReportingRequirements'])['Percentage'].transform(max)
summary_file_rec['Real_Strategy'] = summary_file_rec.groupby(['Typename', 'FinancialReportingRequirements'])['Real_Strategy'].transform('first')
summary_file_rec['Strat_Reccomendation'] = summary_file_rec[summary_file_rec.Percentage.values == summary_file_rec.Real_Strategy.values]["BUProposedStrategytoMeetRequirements"]  
summary_file_rec['Strat_Reccomendation'] = summary_file_rec.groupby(['Typename', 'FinancialReportingRequirements'])['Strat_Reccomendation'].transform('first')
summary_file_rec.loc[summary_file_rec["BUProposedStrategytoMeetRequirements"] == "attribute not required", "Strat_Reccomendation"] = "Attribute not required, no further action"
summary_file_rec["Strat_Reccomendation"].fillna("No strategy on how to send this attribute, across all feeds sending this file format", inplace=True)

summary_file_rec

,Reqcategory,Typename,FinancialReportingRequirements,BUProposedStrategytoMeetRequirements,Feed Count,Percentage,Strat_Percent_Max,Real_Strategy,Strat_Reccomendation
1653,ifrs 17,aeb,"account: claims (e.g. death, annuity, etc.)",attribute not required,2.0,100,100.000000,NaN,"Attribute not required, no further action"
1654,ifrs 17,aeb,account: directly attributable acquisition costs,attribute not required,1.0,50,50.000000,50.000000,"Attribute not required, no further action"
1655,ifrs 17,aeb,account: directly attributable acquisition costs,derived from feed attributes,1.0,50,50.000000,50.000000,attribute not required
1656,ifrs 17,aeb,account: investment component (how you identif...,attribute not required,2.0,100,100.000000,NaN,"Attribute not required, no further action"
1657,ifrs 17,aeb,"account: premiums (first year, renewal, single)",attribute not required,2.0,100,100.000000,NaN,"Attribute not required, no further action"
1658,ifrs 17,aeb,cash flow indicator (cash / accrual / adjustme...,attribute not required,1.0,50,50.000000,50.000000,"Attribute not required, no further action"
1659,ifrs 17,aeb,cash flow indicator (cash / accrual / adjustme...,derived from feed attributes,1.0,50,50.000000,50.000000,attribute not required
1660,ifrs 17,aeb,cohort / group,attribute not required,1.0,50,50.000000,50.000000,"Attribute not required, no further action"
1661,ifrs 17,aeb,cohort / group,derived from feed attributes,1.0,50,50.000000,50.000000,attribute not required
1662,ifrs 17,aeb,conversions: indicator that policy is result o...,attribute not required,2.0,100,100.000000,NaN,"Attribute not required, no further action"


In [241]:
BG_view = df[["BG", "BU", "SourceSystem", "GL Major Product", "PRKey", "Typename", "FinancialReportingRequirements", "Reqcategory"]]
BG_merged_view = pd.merge(BG_view,summary_file_rec,on=["Reqcategory", "Typename", "FinancialReportingRequirements"])
BG_merged_view.drop("Real_Strategy", axis=1, inplace=True)
BG_merged_view

BG_BU_derived = pd.merge(BG_view,df_merge_col_file_rec,on=["Reqcategory", "Typename", "FinancialReportingRequirements"])
BG_BU_derived

,BG,BU,SourceSystem,GL Major Product,PRKey,Typename,FinancialReportingRequirements,Reqcategory,BUProposedStrategytoMeetRequirements,Proposed_Attributes_To_Derive,Count,Total_Derived,Percentage,Derived_Percent_Max,Majority_Attributes_To_Derive
0,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
1,canada,group benefits,oasis,"group life non-par, group health, individual h...",gf.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
2,canada,group benefits,chess,"cdn individual healthy, cdn group health non-i...",g3.txt - disability,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
3,canada,group benefits,chess,"cdn individual healthy, cdn group health non-i...",g3.txt - disability,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
4,canada,group benefits,chess,"cdn individual healthy, cdn group health non-i...",g3.txt - medical dental,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
5,canada,group benefits,chess,"cdn individual healthy, cdn group health non-i...",g3.txt - medical dental,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
6,us,group benefits,oasis,"group life par, group life non-par, group heal...",gu.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
7,us,group benefits,oasis,"group life par, group life non-par, group heal...",gu.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
8,us,group benefits,chess,"us group health non-par, ltd, (sloc, slic)",g2.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"gross benefit, federal tax, provincial tax, de...",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."
9,us,group benefits,chess,"us group health non-par, ltd, (sloc, slic)",g2.txt,new gb ebf,cash flow indicator (cash / accrual / adjustme...,ifrs 17,derived from feed attributes,"pas system identifier, , , , , , .0, .0, .0, .0",1,2,50.000000,50.000000,"gross benefit, federal tax, provincial tax, de..."


In [242]:
# Writes all tables to the excel

file_types_all.to_excel(writer, sheet_name='File Types_by BU')
# Overall analysis by attribute
bu_strategy.to_excel(writer, sheet_name='IFRS 17 Decided_By BU')
# Break down of analysis by attribute
strat_derived_dtype.to_excel(writer, sheet_name='Detailed Analysis_by Attrb')
df_merge_col.to_excel(writer, sheet_name='Strat for Derived_by Attrb')
data_definitions.to_excel(writer, sheet_name='Data Type Breakdown_by Attrb')
# Break down of analysis by file type
strat_and_derived_file.to_excel(writer, sheet_name='Detailed Analysis_by File Type')
df_merge_col_file.to_excel(writer, sheet_name='Strat for Derived_by File Type')


In [257]:
# Creates excel for the BU recommendations
writer_assessments = pd.ExcelWriter('All_BG_Assessments.xlsx', engine='xlsxwriter')

In [258]:
# Writes tables to excel for the BU recommendations.
# Isabelle: these last 2 blocks are a little buggy, and it takes a while for the tables to write. I recommend running these
# 2 blocks twice in order, and then waiting a minute or 2 for the excel to be created. If it doesn't show up in your folders
# after running each block twice, run just these 2 again

BGs = BG_merged_view.BG.unique()
BUs = BG_merged_view.BU.unique()

for BG in BGs:
    for BU in BUs:
        BG_merged_view_BU = BG_merged_view[(BG_merged_view["BG"] == BG) & (BG_merged_view["BU"] == BU)]
        BG_BU_derived_view = BG_BU_derived[(BG_BU_derived["BG"] == BG) & (BG_BU_derived["BU"] == BU)]
        name = BG[:4] + "_" + BU[:10] + "_Strat"
        name1 = BG[:4] + "_" + BU[:10] + "_derived"
        if not BG_merged_view_BU.empty:
            BG_merged_view_BU.to_excel(writer_assessments, sheet_name=name)
        if not BG_BU_derived_view.empty:
            BG_BU_derived_view.to_excel(writer_assessments, sheet_name=name1)
        